In [83]:
import pandas as pd
import base64
import ast
import re
from azure.storage.blob import BlobServiceClient
import json
import os
from PIL import Image, ImageStat
import matplotlib.pyplot as plt
import io  # Used to convert bytes to a file-like object
import cv2
import numpy as np

#Read CSV file
df = pd.read_csv("labeled_train.csv")

#set up storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=mlfinalexam5505462853;AccountKey=0c40lghglG5/GlNK9yujDQAgo38GKoS2I3DeC/g22hwAEIFANKpmC/TqOpRk4RCT1DbfNiHBFt72+AStB+PfUA==;EndpointSuffix=core.windows.net"
container_name = "publicdata"

#create client
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

In [84]:
print(len(df))

67279


In [87]:
# Assuming `container_client` is already defined
#blobs = container_client.list_blobs()
#for blob in blobs:
#    # Check for both the ".jpg" extension and the "samples/" prefix
#    if blob.name.endswith(".jpg") and blob.name.startswith("samples/"):
#        print(blob.name)
#        blob_client = container_client.get_blob_client(blob.name)
#        blob_data = blob_client.download_blob().readall()  # Using `readall` for simplicity#

#        # Create a BytesIO stream with the image data
#        image_data = io.BytesIO(blob_data)

        # Open the image
#        image = Image.open(image_data)
        
#        # Convert to grayscale for brightness calculation
#        grayscale_image = image.convert("L")

        # Calculate the average brightness
#        pixel_values = np.array(grayscale_image)
#        average_brightness = np.mean(pixel_values)

#        print(f"Average Brightness of {blob.name}: {average_brightness}")
        
#        average_contrast = np.std(pixel_values)
#        
#        print(f"Average contrast of {blob.name}: {average_contrast}")
        
        # Display the image using matplotlib
#        plt.imshow(image)
#        plt.axis('off')  # Hide axes for better visualization
#        plt.show()

In [89]:
# Function to calculate average brightness
def calculate_brightness(image):
    grayscale_image = image.convert("L")
    pixel_values = np.array(grayscale_image)
    return np.mean(pixel_values)

# Function to calculate contrast
def calculate_contrast(image):
    grayscale_image = image.convert("L")
    stat = ImageStat.Stat(grayscale_image)
    return stat.stddev[0]

# Create a list to store filenames that need to be removed
files_to_remove = []

# Process each blob
blobs = container_client.list_blobs()
for blob in blobs:
    if blob.name.endswith(".jpg") and blob.name.startswith("samples/"):
        print(f"Processing {blob.name}")
        blob_client = container_client.get_blob_client(blob.name)
        blob_data = blob_client.download_blob().readall()
        image_data = io.BytesIO(blob_data)
        image = Image.open(image_data)

        # Calculate brightness and contrast
        brightness = calculate_brightness(image)
        contrast = calculate_contrast(image)

        print(f"Brightness: {brightness}, Contrast: {contrast}")

        # Check if the image meets the removal criteria
        if brightness < 60 or contrast < 30:
            files_to_remove.append(blob.name)

# Remove rows from the DataFrame
filtered_df = df[~df['filename'].isin(files_to_remove)]

# Save the filtered DataFrame back to the CSV file
filtered_df.to_csv("cleaned_preprocessed.csv", index=False)

Processing samples/CAM_BACK/n003-2018-01-03-12-03-23+0800__CAM_BACK__1514952316316487.jpg
Brightness: 101.80742916666667, Contrast: 53.962410950964895
Processing samples/CAM_BACK/n003-2018-01-08-11-30-34+0800__CAM_BACK__1515382745757583.jpg
Brightness: 103.03134166666666, Contrast: 57.394012278962585
Processing samples/CAM_BACK/n003-2018-07-12-15-40-35+0800__CAM_BACK__1531383596387543.jpg
Brightness: 108.89841805555555, Contrast: 61.68600053224491
Processing samples/CAM_BACK/n005-2018-06-14-20-11-03+0800__CAM_BACK__1528979868187591.jpg
Brightness: 25.917290972222222, Contrast: 26.339793409508985
Processing samples/CAM_BACK/n008-2018-06-04-16-30-00-0400__CAM_BACK__1528144232887570.jpg
Brightness: 114.32514097222223, Contrast: 37.80770588544624
Processing samples/CAM_BACK/n009-2018-05-08-15-52-41-0400__CAM_BACK__1525809758643363.jpg
Brightness: 117.20806944444445, Contrast: 43.433827624402255
Processing samples/CAM_BACK/n010-2018-08-27-12-00-23+0800__CAM_BACK__1535342963787283.jpg
Bright

In [90]:
print(f"Files removed: {files_to_remove}")
print(f"Remaining rows in CSV: {len(filtered_df)}")

Files removed: ['samples/CAM_BACK/n005-2018-06-14-20-11-03+0800__CAM_BACK__1528979868187591.jpg', 'samples/CAM_FRONT_RIGHT/n005-2018-06-14-20-11-03+0800__CAM_FRONT_RIGHT__1528978813270418.jpg']
Remaining rows in CSV: 67277
